# Travail pratique CPH409

## Importation des fonctions

La première étape est d'importer les fonctions nécessaires à nos calculs. L'utilisation de [PySCF](https://pyscf.org/) s'avère utile pour traiter des molécules au niveau de la chimie quantique.

Du module `pyscf`, nous importons
* `gto`: Pour définir une molécule sur un ensemble de base gaussien (*Gaussian type orbitals*);
* `scf`: *Self-consistent field* pour Hartree-Fock;
* `tools`: Divers outils, nous l'utiliserons pour créer des fichiers `.cube` pour visualiser des orbitales moléculaires.

In [1]:
from pyscf import gto, scf, tools

## Définition de la molécule

La première étape est de définir la molécule. La variable `xyz` contient les coordonnées (x, y et z) de chaque atome. Nous créons ensuite une molécule avec la fonction `gto.Mole`, qui accepte les coordonnées des atomes, la charge totale, le spin (différence entre le nombre d'électrons alpha et beta) et l'ensemble de base. La commande `mol.build` construit les quantités nécessaires pour les calculs subséquents.

La cellule suivante contruit un objet python pour la molécule du dihydrogène (neutre, singlet, ensemble de base minimal STO-3G).

In [2]:
xyz = """
H  0.000000    0.000000    0.000000
H  0.000000    0.000000    0.750000
"""

mol = gto.Mole(
    atom = xyz,
    charge=0,
    spin=0,
    basis = "sto3g"
)
mol.build()

Notons que nous utilisons ici un ensemble de base dit *minimal*. 

5 atomes de carbone: 5 * (1s + 2s + 2px + 2py + 2pz) = 25 orbitals atomiques

5 atomes d'hydrogène: 5 * (1s) = 5 orbitals atomiques

30 orbitals atomiques = 30 orbitals moléculaires

## Calcul en Hartree-Fock

L'appel au module `scf.RHF` rend accessible la fonction d'onde approximée avec la méthode Hartree-Fock, ou plus précisément la solution *restricted Hartree-Fock*. Le mot *restricted* fait référence à la méthode qui force les électrons à être pairés (2 électrons par orbitale moléculaire). D'autres méthodes peuvent être qualifiées de `unrestricted`, où cette contrainte n'est pas imposée. C'est à la ligne `hf.kernel()` où le calcul se produit et que le résultat est affiché.

In [3]:
hf = scf.RHF(mol)
hf.kernel()

converged SCF energy = -1.1161514489386


-1.1161514489386022

Le résultat de `-1.11615` est l'énergie de la molécule en hartree (~627.5 kcal/mol). L'appel à la fonction `analyze` imprime diverses quantités utiles:
* Les orbitals moléculaires (énergie et nombre d'électrons);
* Les charges sur chaque atome;
* Le moment dipolaire de la molécule.

In [4]:
hf.analyze();

**** MO energy ****
MO #1   energy= -0.574436558274987 occ= 2
MO #2   energy= 0.660910051285004  occ= 0
 ** Mulliken atomic charges  **
charge of  0H =     -0.00000
charge of  1H =      0.00000
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000,  0.00000


## Calcul avec la théorie de la fonctionelle de la densité

In [5]:
dft = mol.KS()
dft.xc = "b3lyp"
dft.kernel()

converged SCF energy = -1.15833926707312


-1.1583392670731207

In [6]:
dft.analyze();

**** MO energy ****
MO #1   energy= -0.407613047459251 occ= 2
MO #2   energy= 0.421762399249165  occ= 0
 ** Mulliken atomic charges  **
charge of  0H =      0.00000
charge of  1H =      0.00000
Dipole moment(X, Y, Z, Debye):  0.00000,  0.00000,  0.00000


## Analyze/visualisation des orbitales moléculaires

blabla orbital HF sont un peu mieux que DFT etc.

In [32]:
tools.dump_mat.dump_mo(mol, hf.mo_coeff)

               #0        #1       
  0 H 1s       0.54993   1.20096
  1 H 1s       0.54993  -1.20096


In [ ]:
for i in range(hf.mo_coeff.shape[1]):
    tools.cubegen.orbital(mol, f"orbitale_{i+1:02d}.cube", hf.mo_coeff[:,i])